# Finite differences

This exercise aims at solving the Poisson equation

\begin{align}
    -\Delta u &= 10\pi^2 \sin(10\pi x)\cos(20\pi y)&&\text{in}&\Omega&=(0,1)^2,\\
    u(x,y) &= u_0(x) = \sin(\pi x) && \text{on}& \partial\Omega,
\end{align}

with the finite difference scheme on a rectangular grid.
To discretize the equation, use the 5-point-star

\begin{align}
    \frac{1}{h^2} \begin{bmatrix} & -1 & \\ -1 & 4 & -1 \\ & -1 & \end{bmatrix}.
\end{align}

The individual subtasks treat parts of the complete program.

*Hint:* To test your program, you can first of all consider the problem

\begin{align}
    -\Delta u &= 0&&\text{in}&\Omega&=(0,1)^2,\\
    u(x,y) &= x && \text{on}& \partial\Omega,
\end{align}

which has the simple solution $u(x,y) = x$.

The following input statements should be sufficient for your program:

In [19]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve

Start by choosing a resolution `n` and define the right hand side `f` as well as the boundary condition `u0`:

In [28]:
n = 4

#f = lambda x: 10 * np.pi**2 * np.sin(10*np.pi*x[0]) * np.cos(20*np.pi*x[1])
#u0 = lambda x: np.sin(np.pi * x[0])
f = lambda x: 2
u0 = lambda x: x[0]

We now define the grid:

In [21]:
xs, h = np.linspace(0, 1, n, retstep=True)
ys = np.linspace(0, 1, n)

`N` is the number of degrees of freedom, including the points that are fixed by the boundary condition:

In [22]:
N = n*n
N

16

## Task 1

Implement a function `index(i,j)`, which assigns to each pair `(i,j)` with $0\leq\,$ `i`, `j` < `n` a number `n(i,j)`$\in\{0,\dots,N-1\}$ bijectively. This way, the degrees of freedom are continuously numbered.

Implement a function `coord(i,j)`, which assigns to a `(i,j)` with $0\leq\,$ `i`, `j` < `n` the corresponding coordinate `x(i,j)`$\in[0,1]^2$.

In [23]:
def index(i, j):
    return n*i+j;

def coord(i, j):
    return xs[i],ys[j];

## Task 2

Create a sparse matrix `A` as `lil_matrix` of appropriate size, and create a vector `b` of appropriate size that is initialized with 0. Take a look at the [scipy documentation on `lil_matrix`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.lil_matrix.html) for information on how to create sparse matrices.

In [44]:
A = sparse.lil_matrix(np.zeros([N,N]));
b = np.zeros(N);
print(A);
print(b);



[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


Iterate over the inner rows `i` and inner columns `j` and fill the row of the system matrix `A` (according to the 5-point-star formula) and right hand side `b` that corresponds to the degree of freedom `n(i,j)`. Make use of the `index`- and `coord`-methods from task 1.

In [45]:
#fill b
for i in range(1,n-1):
    for j in range(1,n-1):
        b[index(i,j)]=f(coord(i,j));
print(b);
#fill A
for i in range(1,n-1):
    for j in range(1,n-1):
        print(index(i,j));
        A[index(i,j),index(i,j)]= 4;
        A[index(i,j),index(i,j-1)]= -1;
        A[index(i,j),index(i,j+1)]= -1;
        A[index(i,j),index(i+1,j)]= -1;
        A[index(i,j),index(i-1,j)]= -1;
print(A);
        
        



[0. 0. 0. 0. 0. 2. 2. 0. 0. 2. 2. 0. 0. 0. 0. 0.]
5
6
9
10
  (5, 1)	-1.0
  (5, 4)	-1.0
  (5, 5)	4.0
  (5, 6)	-1.0
  (5, 9)	-1.0
  (6, 2)	-1.0
  (6, 5)	-1.0
  (6, 6)	4.0
  (6, 7)	-1.0
  (6, 10)	-1.0
  (9, 5)	-1.0
  (9, 8)	-1.0
  (9, 9)	4.0
  (9, 10)	-1.0
  (9, 13)	-1.0
  (10, 6)	-1.0
  (10, 9)	-1.0
  (10, 10)	4.0
  (10, 11)	-1.0
  (10, 14)	-1.0


## Task 3

Implement a function `dirichlet(i,j)`, which writes the value 1 onto the diagonal of the row of the system matrix that corresponds to the degree of freedom given by `n(i,j)`. Furthermore, the function should write the correct boundary value into the entry with number `n(i,j)` of the right hand side vector. Again, use the functions `index` and `coord` from task 1.

In [ ]:
def dirichlet(i, j):
    A[index(i,j)][index(i,j)] = 1;
    b[index(i,j)]= 

Iterate over the outer vertices and call the `dirichlet` function for each of these vertices.

In [ ]:
# YOUR CODE HERE

## Task 4

Print the memory consumption of the sparse matrix and compare it to the memory consumption of a dense matrix of the same size. It is enough to consider only the memory consumption of the numbers in the matrix themselves (8 bytes for a floating-point number in double-precission).

*Hint:* `A.nnz` provides the number of entries of a sparse matrix that are not 0 (<b>n</b>umber of <b>n</b>on-<b>z</b>eroes).

In [ ]:
# YOUR CODE HERE

How does the memory consumption increase depending on `n`?

YOUR ANSWER HERE

## Task 5

Use the following code to examine the structure of the system matrix `A` for `n`$\leq 32$. Try different values for `n` at the top of the notebook. Do you see a pattern?

In [ ]:
if A.shape[0] <= 32*32:
    Adense = np.array(abs(A).todense())
    plt.figure(figsize=(10, 10))
    plt.pcolormesh(Adense, cmap=plt.cm.gray_r)
    plt.show()

YOUR ANSWER HERE

## Task 6

Convert `A` into a `csr_matrix` and solve the system of equations $Ax=b$ (search in the scipy documentation for how to do this). The solution should be called `x`.

In [ ]:
# YOUR CODE HERE

## Task 7

Use the following code to visualize the solution. Does it look as you would expect? Describe why the result fits or does not fit your expectations. Run the whole notebook for `n`$\,=256$ as well to increase the resolution (you can also try different values for `n` to examine the behavior of the solution regarding the discretization size).

In [ ]:
zs = np.zeros((n, n))
for i in range(0, n):
    for j in range(0, n):
        zs[i,j] = x[index(i, j)]

plt.figure(figsize=(10,10))
plt.pcolormesh(ys, xs, zs)
plt.show()

YOUR ANSWER HERE